# Performing a climatology ESP forecast for a given catchment with already-processed weather timeseries.

This notebook shows how to perform a climatological Extended Streamflow Prediction forecast, using historical weather as a proxy for future weather. Generates the forecasts and then computes evaluation metrics.

In [ ]:
# This entire section is cookie-cutter template to allow calling the servers and instantiating the connection
# to the WPS server. Do not modify this block.
from urllib.request import urlretrieve
from birdy import WPSClient
from example_data import TESTDATA
from pathlib import Path

import os
import datetime as dt
import tempfile
import xarray as xr

# Set environment variable WPS_URL to "http://localhost:9099" to run on the default local server
url = os.environ.get("WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps")
url="http://localhost:9099"
wps = WPSClient(url)
tmp = Path(tempfile.mkdtemp())

In [ ]:
# Define some of the catchment properties. Could also be replaced by a call to the properties WPS!
basin_area = 4523.5
basin_longitude = -72.55
basin_latitude = 48.39
basin_elevation= 300.0

# Choose a hydrological model to use. We have 'HMETS', 'GR4JCN','MOHYSE' and 'HBVEC'.
hydromodel = 'GR4JCN'

# Since we've used GR4JCN, we need to provide that model's parameters.
model_parameters = '0.529, -3.396, 407.29, 1.072, 16.9, 0.947'

# Choose the forecast date of the year (ex. jan-05-2001 will compare the climatology using all jan-05ths from the dataset)
fdate=dt.datetime(2005,10,13)

# The dataset to use to get the forecast timeseries:
ts=str(TESTDATA['raven-gr4j-cemaneige-nc-ts'])
duration=30 #Length of the climatological ESP forecast

# launch the climatology ESP.
resp=wps.climatology_esp(fdate,model_parameters,ts,basin_latitude,basin_longitude,hydromodel,basin_elevation,duration,name='test_basin',area=basin_area)

In [ ]:
# The model has run! We can get the response.
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True will retrieve the actual files and copy them locally. 
[forecast] = resp.get(asobj=True)

#Since we requested output objects, we can simply access the forecast object.
forecast.to_netcdf(str(tmp) + '/fcst.nc')

### Now, we can compute the forecast scores such as CRPS for this forecast using the XSkillScore package:

In [ ]:
ds=xr.open_dataset(str(TESTDATA['raven-gr4j-cemaneige-nc-ts']))
q_obs=ds.qobs.sel(time=slice(fdate,fdate+dt.timedelta(days=duration-1)))

# DEBUG: I HAD TO DO THIS BECAUSE 'MEMBER' DIMENSION DOES NOT EXIST AND IT SEEMS IT IS REQUIRED IN LINE 125 OF
# WPS_FORECAST_EVALUATION.PY
q_obs.expand_dims(dim='member',axis=0).to_netcdf(str(tmp) + '/q_obs.nc','w')

In [ ]:
forecast.q_sim.plot.line('b',x='time')
q_obs.plot.line('r')

In [ ]:
# Now let's run XSkillScore
# THIS FAILS
resp=wps.hindcast_evaluation(obs=str(tmp / 'q_obs.nc'),hcst=str(tmp / 'fcst.nc'), obs_var='qobs', hcst_var='q_sim')